### Limpieza de datos en ficheros CSV usando la Command Line and csvkit

El Museo de Arte Moderno es uno de los museos más influyentes en el mundo y han publicado un conjunto de datasets sobre las obras de arte en su colección. Los datasets, como TODOS los que nos encontremos en nuestra vida diaria (en el poco tiempo que tengo como Data Scientist no he visto el primer dataset que no tenga que limpiar) requieren de tratamiento y limpieza.

En este artículo, aprenderemos cómo usar la biblioteca csvkit para adquirir y explorar datos tabulares.

¿Por qué la línea de comandos?
Gran pregunta! Antes de empezar en esto, me la había hecho millones de veces y la respuesta es que cuando se trabaja en entornos de Data Science en la nube (cloud), a veces sólo se tiene acceso a la shell de un servidor. En estas situaciones, el dominio de la línea de comandos es una verdadera ventaja. Igualmente, cuando se trabaja con Big Data y datasets muuuuuuy grandes, es muy eficiente trabajar por esta vía y que nuestra computadora (ordenador) no se guinde!

A medida que nos volvemos más eficientes, el uso de la línea de comandos para algunas tareas de data science es mucho más rápida que escribir un script de Python o un job de Hadoop. 

Por último, la línea de comandos tiene un gran ecosistema de herramientas. Esto hace que ciertos tipos de tareas, especialmente aquellas que involucren múltiples archivos, sean increíblemente fáciles.

Si eres nuevo en la línea de comandos, te recomiendo que consultes este curso: https://www.dataquest.io/subject/the-command-line?.


Csvkit
Csvkit es una biblioteca optimizada para trabajar con archivos CSV. Está escrito en Python, pero la interfaz principal es la línea de comandos. Puede instalar csvkit utilizando pip:

(Nota: para trabajar desde Jupyter Notebook con la línea de comando, debemos preceder nuestras peticiones con el simbolo "!". Esto le indica al notebook que el comando que se ejecuta es de la command line)

In [ ]:
! pip install csvkit

Vas a necesitar la librería para poder seguir con el tutorial.

Data acquisition

El dataset de las obras de arte del MOMA está disponible en el repo del GitHub del museo. Usemos el comando curl para descargar el fichero Artworks.csv desde Github. 

curl es una herramienta que está dentro de la mayoría de los ambientes shell y que permite la trasferencia de data entre servidores. 

El dataset lo podemos conseguir en el Github del museo (https://github.com/MuseumofModernArt/collection). Debemos verificar que tenemos el link en Raw data. Finalmente usamos el operador > para redireccionar la salida de nuestro comando curl a un fichero llamado artworks.csv.

In [13]:
! curl https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv > artworks.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 49.4M  100 49.4M    0     0  7361k      0  0:00:06  0:00:06 --:--:-- 8572k


Verificamos que efectivamente se descargó el fichero que queremos.

In [14]:
! ls -a

.                             abaloneNames.txt
..                            artworks.csv
.DS_Store                     ign.csv
.ipynb_checkpoints            ign.csv.zip
Sample_Submission_Tm9Lura.csv test.csv
Untitled.ipynb                train.csv
Untitled1.ipynb               wine.csv
abalone.csv                   winequality-red.csv


Data exploration

Podemos usar el comando head para mostrar las primeras n líneas de un fichero (10 por defecto). Para mostrar las primeras 3, cualquiera de estos comandos funcionaran:

In [18]:
! head -n 3 artworks.csv

﻿Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
"Ferdinandsbrücke Project, Vienna, Austria, Elevation, preliminary version",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Fractional and promised gift of Jo Carole and Ronald S. Lauder,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0sWyJwIiwiY29udmVydCIsIi1yZXNpemUgMzAweDMwMFx1MDAzZSJdXQ.jpg?sha=137b8455b1ec6167,,,,,,,,,
"City of Music, National Superior Conservatory of Music and Dance, Paris, France, View from interior courtyard",Christian de Portzamparc,7470,

In [19]:
! head -3 artworks.csv

﻿Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
"Ferdinandsbrücke Project, Vienna, Austria, Elevation, preliminary version",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Fractional and promised gift of Jo Carole and Ronald S. Lauder,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0sWyJwIiwiY29udmVydCIsIi1yZXNpemUgMzAweDMwMFx1MDAzZSJdXQ.jpg?sha=137b8455b1ec6167,,,,,,,,,
"City of Music, National Superior Conservatory of Music and Dance, Paris, France, View from interior courtyard",Christian de Portzamparc,7470,

Como la mayoría de los datasets con los que habitualmente se trabaja, podemos ver que la primera fila es el header o cabecera. Este nos indica el nombre de las columnas. 

Aparte de eso, la salida es desordenada y es difícil ver o detallar alguna información adicional. Necesitamos una herramienta que sepa cómo mostrar archivos CSV de una manera legible.

csvlook

Csvlook es una herramienta dentro de csvkit que nos permite mostrar y previsualizar un archivo CSV como una tabla. 
Si escribimos Csvlook artworks.csv mostrará todo el dataset, lo cual no tiene mucho sentido porque nos mostrará TODO, en estos casos siempre es ideal trabajar con pocas filas, que ya nos permite tener una idea del resto del dataset. 
Podemos hacer esto de la siguiente forma: 

In [20]:
! head -5 artworks.csv | csvlook

| ﻿Title                                                                                                        | Artist                   | ConstituentID | ArtistBio                                 | Nationality | BeginDate | EndDate | Gender |  Date | Medium                                                         | Dimensions | CreditLine                                                              | AccessionNumber | Classification | Department            | DateAcquired | Cataloged | ObjectID | URL                                    | ThumbnailURL                                                                                                                  | Circumference (cm) | Depth (cm) | Diameter (cm) | Height (cm) | Length (cm) | Weight (kg) | Width (cm) | Seat Height (cm) | Duration (sec.) |
| ------------------------------------------------------------------------------------------------------------- | ------------------------ | ------------- | -----------------------------

En datasets con campos muy grandes o que tienen mucha información, lo anterior sigue siendo nada atractivo. Debemos buscar una forma de poder ver estos datos y entenderlos ya que sigue siendo dificil hacer una exploración allí. 

Veamos como podemos utilizar csvcut para seleccionar sólo algunas columnas de interés y mostrarlas. csvcut es una herramienta dentro del ToolKit que permite permite cortar y modificar las columnas en el CSV. Primero vamos a enumerar todas las columnas usando el indicador -n:

In [21]:
! csvcut -n artworks.csv

  1: ﻿Title
  2: Artist
  3: ConstituentID
  4: ArtistBio
  5: Nationality
  6: BeginDate
  7: EndDate
  8: Gender
  9: Date
 10: Medium
 11: Dimensions
 12: CreditLine
 13: AccessionNumber
 14: Classification
 15: Department
 16: DateAcquired
 17: Cataloged
 18: ObjectID
 19: URL
 20: ThumbnailURL
 21: Circumference (cm)
 22: Depth (cm)
 23: Diameter (cm)
 24: Height (cm)
 25: Length (cm)
 26: Weight (kg)
 27: Width (cm)
 28: Seat Height (cm)
 29: Duration (sec.)


Podemos usar el flag -c para especificar la o las columnas que queremos.

In [ ]:
! csvcut -c 1,2,3 artworks.csv | csvlook 

In [ ]:
QUEDE AQUI

We can then use the -c flag to specify the columns we want. csvcut -c 1,2,3 artworks.csv | csvlook 
will return the first 3 columns. You can also use the column names themselves: csvcut -c Artist,ArtistBio,Date.

Running either command will display the 3 columns for the entire dataset so we’ll need to take advantage of piping to view just a few lines.

We can consult the csvkit documentation to read about piping between csvkit utilities:

All csvkit utilities accept an input file as “standard in”, in addition to as a filename. This means that we can make the output of one csvkit utility become the input of the next.

This means we can pipe the stdout of csvcut to the stdin of csvlook! We can build the following pipeline:

extract just the first 10 lines using head
filter to just the first 3 columns using csvcut
display in a clean way using csvlook
  
head -10 artworks.csv | csvcut -c 1,2,3 | csvlook

csvgrep
When working with historical datasets, we need to make sure the date and time columns are formatted correctly (or even basic time series plots won’t work). Let’s explore the Date and DateAcquired columns:

  
head -20 artworks.csv | csvcut -c Date,DateAcquired | csvlook

In [ ]:
While the first 20 values in DateAcquired look fine, the Date column has some values that won’t play nicely with most data visualization tools like 1976-77. We can easily deal with this by just selecting the first year in the range (e.g. 1976 from the range 1976-77). Before we do that, let’s figure out how many lines match this pattern.

We can use the csvgrep tool to extract all values in a column (or columns) that match a regular expression. We specify the columns we want csvgrep to match on using the -c flag. We specify the regular expression we want csvgrep to use using the -regex flag.

The regex ^([0-9]*-[0-9]*) matches pairs of numeric values that are separated by a hyphen (-). Since we’re searching for instances of the pattern on the Date column, we write the following:

  
csvgrep --c Date --regex "^([0-9]*-[0-9]*)"
Let’s modify and run the pipeline we’ve built to incorporate csvgrep:

  
head -10 artworks.csv | csvcut -c Date | csvgrep --c Date --regex "^([0-9]*-[0-9]*)" | csvlook

csvstat
Wonderful! Now that we know it works, let’s apply the regular expression over the entire Date column (instead of just the first 10 lines) and determine how many lines match this pattern. The csvstat tool takes a CSV as an input (stdin) and computes summary statistics. We can use the --count flag to specify that we only want the line count. We can also remove csvcut, head, and csvlook since we don’t need to display the output.

  
csvgrep --c Date --regex "^([0-9]*-[0-9]*)" artworks.csv | csvstat --count
Row count: 18073
It looks like 18,073 lines match this pattern. Let’s now calculate:

how many lines match the 4 digit year pattern
how many total lines the dataset contains
We can use the regex (^[0-9]{4}$) to find all 4 digit year values and pipe the results to csvstat:

  
csvgrep --c Date --regex "[0-9]{4}$" artworks.csv | csvstat --count
Row count: 76263
Finally, to get the total number of lines of a dataset, we can use the wc command with the -l flag (to display just the number of `lines):

  
wc -l artworks.csv
137382 artworks.csv
If we combine the number of lines that match the 4 digit year regex (76263) with the number of lines that match the year range regex (18073), we get (94336) lines. Given that there are 137,382 lines total, this is a great starting point for our analysis!

In [ ]:
Ronny De Abreu.
Tomando como referencia: https://www.dataquest.io/blog/data-cleaning-command-line/

In [ ]:
Hacer este mismo con Python --> https://www.dataquest.io/blog/data-cleaning-with-python/